# Face mask detection with alert system

In [2]:
##Importing necessary Libraries
from keras.models import load_model
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import smtplib
import tkinter

In [4]:
import os
os.getcwd()

'C:\\Users\\Arif Khan'

In [5]:
# Initialize Tkinter
root = tkinter.Tk()
root.withdraw()

# Load trained Deep Learning Model
model = load_model('face_mask_detection_with_alert_system.h5')

# Classifier to Detect Face
face_det_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Capture Video
vid_source = cv2.VideoCapture(0)


text_dict = {0:'Mask ON',1:'No Mask'}
rect_color_dict = {0:(0,255,0),1:(0,0,255)}

In [6]:
SUBJECT = "Subject"
TEXT = "One visiter violated face mask policy. See in the camera to recognize user. A Person has been detected without a face mask."

In [ ]:
# While loop to continuously detect camera feed
while(True):

  ret,img = vid_source.read()
  grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  faces = face_det_classifier.detectMultiScale(grayscale_img,1.3,5)

  for (x,y,w,h) in faces:
    face_img = grayscale_img[y:y+w,x:x+w]
    resized_img = cv2.resize(face_img,(112,112))
    normalized_img = resized_img/255.0
    reshaped_img = np.reshape(normalized_img,(1,112,112,1))
    result = model.predict(reshaped_img)

    label = np.argmax(result,axis=1)[0]

    cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
    cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
    cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
    
    ## If label=1 then it means wearing No Mask and 0 means wearing Mask

    if (label==1): 
        
        # throw a warning message to tell user to wear a mask if not wearning one. This will stay open and 
        # No Access will be given.
        
      messagebox.showwarning("Warning","Access Denied. Please Wear a Face Mask")
    
    ## Send a mail to the administrator if access denied/User not wearing face mask 

      message = 'Subject: {}\n\n{}'.format(SUBJECT, TEXT)
      mail = smtplib.SMTP('smtp.gmail.com', 587)
      mail.ehlo()
      mail.starttls()
      mail.login('arifkhan435@gmail.com', '9772878395')
      mail.sendmail('arifkhan435@gmail.com', 'arifkhan435@gmail.com', message)
      mail.close
    else:
      pass
      break

  cv2.imshow('LIVE Video Feed',img)
  key = cv2.waitKey(1)

  if (key==27):
    break

cv2.destroyAllWindows()
source.release()